Set your password in a local 'nira_user' and 'nira_pass' environment variables, e.g.

`export nira_user=egge@highstreetconsulting.com`

`export nira_pass=blahblahblah` (*nix)


## Useful Resources:

Vector Tile Specification: https://github.com/mapbox/vector-tile-spec/blob/master/2.1/README.md

MapTiler Playground: https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/

Mapbox Vector Tile docs: https://github.com/tilezen/mapbox-vector-tile

Mercantile: https://mercantile.readthedocs.io/en/latest/api/mercantile.html#module-mercantile


In [1]:
import mercantile
import os
import requests
import mapbox_vector_tile
import math
import json

base_url = 'https://gui.cloud.niradynamics.se/api2018'

In [2]:
# given the x, y, z descriptor of a tile and the x, y pixel coordinates, convert
# to lat/lon decimal degrees
# converts from MVT encoding to WGS84 decimal degrees
def pixel2deg(xtile, ytile, zoom, x, y, extent = 4096):
  n = 2.0 ** zoom
  xtile = xtile + (x / extent)
  ytile = ytile + ((extent - y) / extent)
  lon_deg = (xtile / n) * 360.0 - 180.0
  lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
  lat_deg = math.degrees(lat_rad)
  return (lon_deg, lat_deg)

In [3]:
# get token
endpoint = '/user/login'
data = {
  "username": os.environ['nira_user'],
  "password": os.environ['nira_pass']
}

url = base_url + endpoint

r = requests.post(url, json=data)
cookies = r.cookies

In [10]:
# get tile - example
x, y, z = (771, 1644, 12) # note - differs from order in Nira API 12/757/1635'

url = base_url + '/tiles/vector/aggregated/roughness-long-term-aggregation/{}/{}/{}'.format(z, x, y)
r = requests.get(url, cookies=cookies)
assert r.status_code == 200

mvt = mapbox_vector_tile.decode(r.content, transformer = lambda px, py: pixel2deg(x, y, z, px, py))

# remove local roads
features = mvt['nira-roughness-long-term-aggregation']['features']
features = [f for f in features if f['properties']['FunctionalRoadClass'] <= 4]
mvt['nira-roughness-long-term-aggregation']['features'] = features

with open('test.json', 'w') as outfile:
    json.dump(mvt, outfile)

In [11]:
# get tiles for sun cloud bounding box
tiles = mercantile.tiles(-113.426177, 31.334485, -109.051065, 34.044281, 12)
tile_list = [t for t in tiles]
len(tile_list)
rejects = []
empty = []

1938

In [16]:
for tile in tile_list:
    out_file_path = 'out_tiles/tile_{}_{}_{}.json'.format(tile.z, tile.x, tile.y)
    if os.path.exists(out_file_path):
        print('Skipping existing file {}'.format(out_file_path))
        continue

    if tile in rejects:
        print('Skipping reject {}'.format(out_file_path))
        continue
    
    # note - differs from order in Nira API
    print('Fetching {} {}'.format(tile.x, tile.y))
    url = base_url + '/tiles/vector/aggregated/roughness-long-term-aggregation/{}/{}/{}'.format(tile.z, tile.x, tile.y)
    r = requests.get(url, cookies=cookies)
    if not r.status_code == 200:
        rejects.append(tile)
        print('Rejected')
        continue

    mvt = mapbox_vector_tile.decode(r.content, transformer = lambda x, y: pixel2deg(tile.x, tile.y, tile.z, x, y))

    # remove local roads
    features = mvt['nira-roughness-long-term-aggregation']['features']
    features = [f for f in features if f['properties']['FunctionalRoadClass'] <= 4]

    if len(features) == 0:
        empty.append(tile)
        print('Tile {} {} Empty'.format(tile.x, tile.y))
        continue
        
    mvt['nira-roughness-long-term-aggregation']['features'] = features

    with open(out_file_path, 'w') as outfile:
        json.dump(mvt, outfile)

Skipping reject out_tiles/tile_12_757_1635.json
Skipping reject out_tiles/tile_12_757_1636.json
Skipping existing file out_tiles/tile_12_757_1637.json
Skipping existing file out_tiles/tile_12_757_1638.json
Skipping existing file out_tiles/tile_12_757_1639.json
Skipping existing file out_tiles/tile_12_757_1640.json
Skipping existing file out_tiles/tile_12_757_1641.json
Skipping existing file out_tiles/tile_12_757_1642.json
Skipping existing file out_tiles/tile_12_757_1643.json
Skipping reject out_tiles/tile_12_757_1644.json
Skipping reject out_tiles/tile_12_757_1645.json
Skipping reject out_tiles/tile_12_757_1646.json
Skipping reject out_tiles/tile_12_757_1647.json
Skipping reject out_tiles/tile_12_757_1648.json
Skipping reject out_tiles/tile_12_757_1649.json
Skipping reject out_tiles/tile_12_757_1650.json
Skipping existing file out_tiles/tile_12_757_1651.json
Skipping existing file out_tiles/tile_12_757_1652.json
Skipping existing file out_tiles/tile_12_757_1653.json
Skipping reject ou

# Post Processing

After running the scraping code, open QGIS and use the `Merge Vector Layers` tool to combine all of the .json files into a single file. Optionally, select features where functional class >= 4 (I think) and export to a geopackage.